Heatmap generation

In [ ]:
import cv2
import numpy as np
import random

# Open video file
video_path = 'cropped_EyeTracking.mp4'
cap = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Cannot open video file.")
    exit()

# Get video dimensions
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Get video frame rate
fps = cap.get(cv2.CAP_PROP_FPS)
if fps == 0:
    fps = 30  # Set a default frame rate if retrieval fails

frame_interval = int(1000 / fps)
frames_to_show = int(0.5 * fps)  # Frames to display each point (0.5 seconds)

# Simulate gaze points (for demonstration)
num_points = 200
gaze_points = [(random.randint(0, width), random.randint(0, height)) for _ in range(num_points)]

# Initialize frame counter and heatmap list
frame_count = 0
heatmap_list = []  # To store the heatmaps for active points

# Loop through video frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Add new gaze point heatmap every frames_to_show frames
    if frame_count < len(gaze_points):
        point = gaze_points[frame_count]
        heatmap = np.zeros((height, width), dtype=np.float32)
        cv2.circle(heatmap, point, 30, 1.0, thickness=-1)  # Adjust radius for heat intensity
        heatmap_list.append((heatmap, frames_to_show))  # Store heatmap with display duration

    # Composite heatmaps and apply fading effect
    combined_heatmap = np.zeros((height, width), dtype=np.float32)
    for i, (heatmap, remaining_frames) in enumerate(heatmap_list):
        if remaining_frames > 0:
            heatmap_list[i] = (heatmap, remaining_frames - 1)  # Decrease remaining frames
            combined_heatmap += heatmap * (remaining_frames / frames_to_show)  # Fade effect

    # Apply Gaussian blur to the combined heatmap to smooth it out
    heatmap_blurred = cv2.GaussianBlur(combined_heatmap, (0, 0), sigmaX=25, sigmaY=25, borderType=cv2.BORDER_DEFAULT)

    # Normalize the heatmap to the range [0, 255]
    heatmap_normalized = cv2.normalize(heatmap_blurred, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)

    # Apply a colormap to convert the grayscale heatmap to a color heatmap
    heatmap_colored = cv2.applyColorMap(heatmap_normalized.astype(np.uint8), cv2.COLORMAP_JET)

    # Overlay the heatmap on the original frame
    overlay = cv2.addWeighted(frame, 0.7, heatmap_colored, 0.3, 0)

    # Display the video with the heatmap overlay
    cv2.imshow('Video with Gaze Heatmap', overlay)

    # Exit if 'q' is pressed
    if cv2.waitKey(frame_interval) & 0xFF == ord('q'):
        break

    # Increase frame count
    frame_count += 1

    # Remove heatmaps that have fully faded out
    heatmap_list = [(heatmap, remaining_frames) for heatmap, remaining_frames in heatmap_list if remaining_frames > 0]

# Release resources
cap.release()
cv2.destroyAllWindows()


Step 0: Images resize from 2560 * 720 to 1080 * 360

In [ ]:
import cv2
import os

########################################################
# GCD of image resolution (2560 x 720) is: 80
# 9.0
# 32.0
########################################################

# Define the directory where the images are stored
image_dir = "images_GCD2"
output_dir = "images_GCD4_1080_360"  # Where resized images will be saved

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Define new dimensions
new_width = 1080
new_height = 360

# Loop through the images
for index in range(720, 746):  # Assuming index ranges from 1 to 100
    # Construct the image file name
    image_path = os.path.join(image_dir, f"frame_{index}.png")
    output_path = os.path.join(output_dir, f"frame_{index}.png")

    # Check if the image file exists
    if not os.path.isfile(image_path):
        print(f"Warning: {image_path} does not exist.")
        continue

    # Load the image using OpenCV
    img = cv2.imread(image_path)

    # Check if the image was loaded successfully
    if img is None:
        print(f"Error: Could not load {image_path}.")
        continue

    # Resize the image to half its original size
    img_resized = cv2.resize(img, (img.shape[1] // 2, img.shape[0] // 2), interpolation=cv2.INTER_AREA)
    
    # Resize the image to 1080x360 using cv2.resize()
    img_resized = cv2.resize(img_resized, (new_width, new_height))

    # Save the resized image to the output directory
    cv2.imwrite(output_path, img_resized)

    # print(f"Resized {image_path} to {img_resized.shape[1]}x{img_resized.shape[0]} and saved to {output_path}")


Scoring for the 1080 * 360 images


Step 1: Detect bounding box + save bounding_boxes_GCD4_1080_360. csv + save processed images with bounding box

In [1]:
# Input: 
# - raw images 1080*360 pixels
# - range
# output: 
# - bounding_boxes_GCD4_1080_360.csv [index, x1, y1, x2, y2, confidence, class_id]
# - processed_frame_{index}.png

import os
import csv
import cv2  # Ensure you have OpenCV imported
from ultralytics import YOLO

# Define the directory where the images are stored
image_dir = "images_GCD4_1080_360"
output_dir = "images_GCD4_1080_360"

# Load the YOLO model
model = YOLO("yolov8n.pt")  # Use the path to your YOLO model, e.g., "yolov8n.pt"

# Define the output CSV file
output_csv = "bounding_boxes_GCD4_1080_360.csv"

# Open the CSV file for writing
with open(output_csv, mode='w', newline='') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(["Image", "x1", "y1", "x2", "y2", "Confidence", "Class"])

    # Loop through the images
    for index in range(720, 746):  # Assuming index ranges from 720 to 745
        # Construct the image file name
        image_path = os.path.join(image_dir, f"frame_{index}.png")

        # Check if the image file exists
        if not os.path.isfile(image_path):
            print(f"Warning: {image_path} does not exist.")
            continue

        # Perform object detection on the image
        # results = model(image_path, show=False)  # 'show=False' if you don't want to display the image
        results = model(image_path, show=False, conf = 0.1) 

        # Save the processed image (getting the first image from results)
        processed_image = results[0].orig_img  # Get the original image after processing

        # Extract and draw bounding box coordinates on the image
        for result in results:  # Iterate over the detection results
            for box in result.boxes:  # Iterate over each box in the results
                # Get bounding box coordinates, confidence, and class
                x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())  # Convert coordinates to integers
                confidence = box.conf[0].item()  # Confidence score
                class_id = int(box.cls[0].item())  # Class ID as an integer

                # Draw the bounding box on the image
                cv2.rectangle(processed_image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green box, thickness 2
                label = f"Class: {class_id}, Conf: {confidence:.2f}"
                cv2.putText(processed_image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                # Write the data to the CSV file
                writer.writerow([index, x1, y1, x2, y2, confidence, class_id])
                # writer.writerow([f"frame_{index}.png", x1, y1, x2, y2, confidence, class_id])

        # Save the processed image with bounding boxes
        output_path = os.path.join(output_dir, f"processed_frame_{index}.png")
        cv2.imwrite(output_path, processed_image)  # Save the image with the bounding boxes drawn

print(f"Bounding boxes saved to {output_csv}")



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/jhpark/miniforge3/envs/baar/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/jhpark/miniforge3/envs/baar/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/Users/jhpark/miniforge3/envs/baar/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/jhpark/miniforge3/envs/baar/lib/python3.9/site-packages/traitlets/config/application.py", line 1075, 

RuntimeError: Numpy is not available

Step 2: Make the new merged bounding box (merging overlapping or close boxes), map the coordinates of the bounding boxes to the key, save the dictionary

In [ ]:
# Input: 
# - bounding_boxes_GCD4_1080_360.csv [index, x1, y1, x2, y2, confidence, class_id]
# - range
# output: 
# - merged_frame_{image_index}.png in "/merged_bounding_boxes_images_GCD4_1080_360"
# - merged_bounding_boxes.pkl [frame_index*, merged_bounding_box_position (x1 y1 x2 y2)]

import numpy as np
import cv2
import csv
import os  # Add this line to import the os module
import pickle

def get_bounding_box_center(x1, y1, x2, y2):
    """Calculate the center of a bounding box."""
    center_x = (x1 + x2) / 2
    center_y = (y1 + y2) / 2
    return center_x, center_y

def boxes_overlap(box1, box2):
    """Check if two boxes overlap."""
    x1_1, y1_1, x2_1, y2_1 = box1
    x1_2, y1_2, x2_2, y2_2 = box2
    
    if x1_1 > x2_2 or x1_2 > x2_1:
        return False
    if y1_1 > y2_2 or y1_2 > y2_1:
        return False
    return True

def merge_boxes(boxes, edge_threshold=50, range_threshold=50):
    """Merge all overlapping bounding boxes."""
    merged_boxes = []
    used_boxes = [False] * len(boxes)
    
    for i, box1 in enumerate(boxes):
        if used_boxes[i]:
            continue
        
        merged_box = box1
        used_boxes[i] = True
        merged = True
        
        while merged:
            merged = False
            for j, box2 in enumerate(boxes):
                if used_boxes[j]:
                    continue
                    
                if boxes_overlap(merged_box, box2):
                    merged_box = [
                        min(merged_box[0], box2[0]),
                        min(merged_box[1], box2[1]),
                        max(merged_box[2], box2[2]),
                        max(merged_box[3], box2[3])
                    ]
                    used_boxes[j] = True
                    merged = True
                
                # Check for close but non-overlapping boxes
                if not used_boxes[j]:
                    x1_1, y1_1, x2_1, y2_1 = merged_box
                    x1_2, y1_2, x2_2, y2_2 = box2

                    horizontal_edge_close = abs(x2_1 - x1_2) < edge_threshold or abs(x1_1 - x2_2) < edge_threshold
                    vertical_edge_close = abs(y2_1 - y1_2) < edge_threshold or abs(y1_1 - y2_2) < edge_threshold

                    if vertical_edge_close:
                        left_right_in_range = abs(x1_1 - x1_2) < range_threshold and abs(x2_1 - x2_2) < range_threshold
                        if left_right_in_range:
                            merged_box = [
                                min(merged_box[0], box2[0]),
                                min(merged_box[1], box2[1]),
                                max(merged_box[2], box2[2]),
                                max(merged_box[3], box2[3])
                            ]
                            used_boxes[j] = True
                            merged = True

                    if horizontal_edge_close:
                        top_bottom_in_range = abs(y1_1 - y1_2) < range_threshold and abs(y2_1 - y2_2) < range_threshold
                        if top_bottom_in_range:
                            merged_box = [
                                min(merged_box[0], box2[0]),
                                min(merged_box[1], box2[1]),
                                max(merged_box[2], box2[2]),
                                max(merged_box[3], box2[3])
                            ]
                            used_boxes[j] = True
                            merged = True

        merged_boxes.append(merged_box)
    
    return merged_boxes

# (1) Load bounding boxes from CSV
csv_file = "bounding_boxes_GCD4_1080_360.csv"
bounding_boxes = {}

with open(csv_file, mode='r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        image_index = row["Image"]
        x1, y1, x2, y2 = map(float, [row["x1"], row["y1"], row["x2"], row["y2"]])
        if image_index not in bounding_boxes:
            bounding_boxes[image_index] = []
        bounding_boxes[image_index].append((x1, y1, x2, y2))

# (2) Merge bounding boxes and store in a dictionary
merged_bounding_boxes = {}

for image_index, boxes in bounding_boxes.items():
    # Perform box merging for each image
    merged_boxes = merge_boxes(boxes)
    merged_bounding_boxes[image_index] = merged_boxes

# (3) Displaying and Saving the results
output_dir = "F:/OneDrive - The University of Texas at Austin/Research/Topic_BAAR/merged_bounding_boxes_images_GCD4_1080_360"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for image_index, merged_boxes in merged_bounding_boxes.items():
    # Load the image
    image_path = f"F:/OneDrive - The University of Texas at Austin/Research/Topic_BAAR/images_GCD4_1080_360/processed_frame_{image_index}.png"  # Modify this path accordingly
    original_img = cv2.imread(image_path)

    # Draw merged bounding boxes on the image
    for (x1, y1, x2, y2) in merged_boxes:
        cv2.rectangle(original_img, (int(x1), int(y1)), (int(x2), int(y2)), (255, 255, 0), 2)  # Cyan color for merged boxes
        # Ensure the coordinates are integers for putText
        x1, y1 = int(x1), int(y1)
        cv2.putText(original_img, "merged", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)

    # Save the result image
    output_image_path = os.path.join(output_dir, f"merged_frame_{image_index}.png")
    cv2.imwrite(output_image_path, original_img)

    # Optionally display the image
    # cv2.imshow(f"Processed frame_{image_index}.png", original_img)
    # cv2.waitKey(0)

# Optionally save merged bounding boxes and frame mappings to a pickle or JSON file
import pickle
with open('merged_bounding_boxes.pkl', 'wb') as f:
    pickle.dump(merged_bounding_boxes, f)

print("Merged bounding boxes saved and processing completed.")


This is a test for the HashMap merged_bounding_boxes.pkl

In [ ]:
import pickle

merged_boxes_file = "merged_bounding_boxes.pkl"

with open(merged_boxes_file, 'rb') as f:
    merged_bounding_boxes = pickle.load(f)

for frame_index, boxes in merged_bounding_boxes.items():
    print(f"Frame Index: {frame_index}")
    for box in boxes:
        x1, y1, x2, y2 = box
        print(f"Bounding Box: x1={x1}, y1={y1}, x2={x2}, y2={y2}")


Step 3: Create Grid, Assign Scores, set detection bounding boxes, and Visualization

For each individual frame, score grid + detection bounding box + save score.csv + save scored images (using coordinate as key)

In [ ]:
# Input: 
# - merged_bounding_boxes.pkl [frame_index*, merged_bounding_box_position x1 y1 x2 y2]
# - range

# output: 
# - grid_scores_1080_360.pkl:
    # grid_scores_dict[coordinates] = {
    #             'frame': index,
    #             'grid_scores': grid_scores,
    #             'bounding_boxes_with_scores': bounding_boxes_with_scores  # Store bounding boxes with score > 0
    #         }
# - "processed_{image_name}": scored images with merged bounding boxes and heatmap 



import os
import cv2
import csv
import numpy as np
import pickle

# Set paths
image_dir = "F:/OneDrive - The University of Texas at Austin/Research/Topic_BAAR/images_GCD4_1080_360"
output_dir = "F:/OneDrive - The University of Texas at Austin/Research/Topic_BAAR/images_GCD4_1080_360_scored"
grid_scores_file = "grid_scores_1080_360.pkl"  # File to save grid scores
frame_data_file = "FrameandData.csv"
merged_boxes_file = "merged_bounding_boxes.pkl"

# Grid parameters
grid_size = 10
img_width = 1080
img_height = 360
grid_width = img_width // grid_size
grid_height = img_height // grid_size

# Dictionary to store grid scores for each frame
grid_scores_dict = {}

# Load frame and coordinate mappings from CSV
frame_to_coordinate = {}
with open(frame_data_file, mode='r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        frame = int(row['Frame'])
        x, y, angle = float(row['X_location(m)']), float(row['Y_location(m)']), float(row['Yaw_angle(deg)'])
        frame_to_coordinate[frame] = (x, y, angle)

# Load merged bounding boxes from the pickle file
merged_bounding_boxes = {}
with open(merged_boxes_file, 'rb') as f:
    data = pickle.load(f)
    for frame_index, boxes in data.items():
        merged_bounding_boxes[frame_index] = boxes

# Process each image based on merged bounding boxes
for index in range(720, 746):  # Adjust the range according to your image set
    image_name = f"frame_{index}.png"
    image_path = os.path.join(image_dir, image_name)
    if not os.path.isfile(image_path):
        print(f"Image {image_name} does not exist. Skipping...")
        continue

    # Load the image
    original_img = cv2.imread(image_path)
    if original_img is None:
        print(f"Failed to load {image_name}. Skipping...")
        continue

    # Initialize grid scores
    grid_scores = np.zeros((grid_height, grid_width))

    # Get bounding boxes for the current image
    boxes = merged_bounding_boxes.get(str(index), [])

    # List to store bounding boxes with scores > 0
    bounding_boxes_with_scores = []

    # Calculate grid scores
    for (x1, y1, x2, y2) in boxes:
        x1_grid = int(x1 // grid_size)
        y1_grid = int(y1 // grid_size)
        x2_grid = int(x2 // grid_size)
        y2_grid = int(y2 // grid_size)

        box_has_score = False  # Flag to track if this bounding box has a score > 0

        for i in range(y1_grid, y2_grid + 1):
            for j in range(x1_grid, x2_grid + 1):
                if i >= 0 and i < grid_height and j >= 0 and j < grid_width:
                    if i == y1_grid or i == y2_grid or j == x1_grid or j == x2_grid:
                        grid_scores[i, j] = max(grid_scores[i, j], 0.6)  # Partially filled grid
                    else:
                        grid_scores[i, j] = 1.0  # Fully filled grid

                    # Check if this grid has a score > 0
                    if grid_scores[i, j] > 0:
                        box_has_score = True

        # Adjacent grid penalty
        for i in range(max(0, y1_grid - 1), min(grid_height, y2_grid + 2)):
            for j in range(max(0, x1_grid - 1), min(grid_width, x2_grid + 2)):
                if grid_scores[i, j] == 0:
                    grid_scores[i, j] = 0.2  # Adjacent to a filled grid

        # If this bounding box had any score > 0, add it to the list
        if box_has_score:
            bounding_boxes_with_scores.append((x1, y1, x2, y2))

    # Normalize the grid scores
    grid_scores = np.clip(grid_scores, 0, 1)

    # Store the grid scores and bounding boxes with scores > 0 in the dictionary using the coordinates as the key
    if index in frame_to_coordinate:
        coordinates = frame_to_coordinate[index]
        grid_scores_dict[coordinates] = {
            'frame': index,
            'grid_scores': grid_scores,
            'bounding_boxes_with_scores': bounding_boxes_with_scores  # Store bounding boxes with score > 0
        }

    # Create a heatmap from grid scores
    heatmap = cv2.resize(grid_scores, (img_width, img_height), interpolation=cv2.INTER_LINEAR)
    heatmap_colored = cv2.applyColorMap((heatmap * 255).astype(np.uint8), cv2.COLORMAP_JET)

    # Overlay the heatmap on the original image with transparency
    overlay = cv2.addWeighted(original_img, 0.7, heatmap_colored, 0.3, 0)

    # Draw bounding boxes on the overlay
    for (x1, y1, x2, y2) in boxes:
        cv2.rectangle(overlay, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

    # Draw grid lines
    for i in range(grid_size, img_width, grid_size):
        cv2.line(overlay, (i, 0), (i, img_height), (255, 255, 255), 1)
    for i in range(grid_size, img_height, grid_size):
        cv2.line(overlay, (0, i), (img_width, i), (255, 255, 255), 1)

    # Save the processed image
    output_path = os.path.join(output_dir, f"processed_{image_name}")
    cv2.imwrite(output_path, overlay)

# Save the grid scores dictionary to a file using pickle
with open(grid_scores_file, 'wb') as f:
    pickle.dump(grid_scores_dict, f)

print("Grid scores and bounding boxes saved to grid_scores_1080_360.pkl")
print("Processing completed.")


This is a test for HashMap "grid_scores_1080_360.pkl" with coordinate, frame, and bounding box

In [ ]:
import pickle

# Load the saved grid scores dictionary
grid_scores_file = "grid_scores_1080_360.pkl"
with open(grid_scores_file, 'rb') as f:
    grid_scores_dict = pickle.load(f)

# Print all contents of grid_scores_dict
for coordinates, data in grid_scores_dict.items():
    print(f"Coordinates: {coordinates}, Frame: {data['frame']}")
    print("Grid scores:")
    print(data['grid_scores'])
    print("bounding box")
    print(data['bounding_boxes_with_scores'])
    print()  # Blank line for better readability


# import pickle

# # Load the pickle file
# with open('grid_scores_1080_360.pkl', 'rb') as file:
#     data = pickle.load(file)

# def get_bounding_boxes_by_frame(data, target_frame):
#     bounding_boxes = []

#     # Iterate over each item in the dictionary
#     for coordinate, details in data.items():
#         if details['frame'] == target_frame:
#             bounding_boxes = details['bounding_boxes_with_scores']
#             break

#     return bounding_boxes

# # Example usage
# target_frame_index = 735
# bounding_boxes = get_bounding_boxes_by_frame(data, target_frame_index)

# print(f"Bounding boxes for frame {target_frame_index}:")
# for bbox in bounding_boxes:
#     print(bbox)



Step 3: (Optional) Transform to a video

In [ ]:
# Input: 
# - scored images with merged bounding boxes and heatmap
# - rangeh
# output: 
# - video

import os
import cv2

# Set the path to the directory where your images are located
image_dir = "F:/OneDrive - The University of Texas at Austin/Research/Topic_BAAR/images_GCD4_1080_360_scored"
output_video = "output_video_GCD4_1080.mp4"

# Define the range of indices for the images you want to include in the video
image_indices = range(720, 746)  # Change the range as per your requirement

# Define video properties
fps = 2  # Frames per second
frame_size = (1080, 360)  # Width and height of the video (match the image resolution)

# Initialize the video writer using the 'XVID' codec
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for widely supported AVI/MPEG-4 format
video_writer = cv2.VideoWriter(output_video, fourcc, fps, frame_size)

# Loop through the image indices
for index in image_indices:
    image_path = os.path.join(image_dir, f"processed_frame_{index}.png")

    # Check if the image exists
    if not os.path.isfile(image_path):
        print(f"Image {image_path} does not exist. Skipping...")
        continue

    # Read the image
    img = cv2.imread(image_path)

    # Resize the image if it's not the same size as the frame size
    if img.shape[1] != frame_size[0] or img.shape[0] != frame_size[1]:
        img = cv2.resize(img, frame_size)

    # Write the image to the video as a frame
    video_writer.write(img)

# Release the video writer
video_writer.release()

print(f"Video saved as {output_video}")

Step 4: Loading bounding box and Cropping the corresponding parts (conservative way: all grids with score>0)

In [ ]:
# assume based on frame to retrive bounding boxes

# Input: 
# - grid_scores_1080_360.pkl:
    # grid_scores_dict[coordinates] = {
    #             'frame': index,
    #             'grid_scores': grid_scores,
    #             'bounding_boxes_with_scores': bounding_boxes_with_scores  # Store bounding boxes with score > 0
    #         }
# - frame_index

# output: 
# - cropped images



import os
import cv2
import pickle

# File paths
image_dir = "F:/OneDrive - The University of Texas at Austin/Research/Topic_BAAR/images_GCD4_1080_360"
grid_scores_file = "grid_scores_1080_360.pkl"
output_dir = "cropped_images"  # Directory to save cropped images

# Create output directory if it does not exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Load grid scores dictionary and bounding boxes
with open(grid_scores_file, 'rb') as f:
    grid_scores_dict = pickle.load(f)

# Function to find bounding boxes based on frame index
def get_bounding_boxes_by_frame(data, target_frame):
    bounding_boxes = []

    # Iterate over each item in the dictionary
    for coordinate, details in data.items():
        if details['frame'] == target_frame:
            bounding_boxes = details['bounding_boxes_with_scores']
            break

    return bounding_boxes

# Function to crop the image based on frame index and bounding box
def crop_image(frame_index, bounding_box):
    image_name = f"frame_{frame_index}.png"
    image_path = os.path.join(image_dir, image_name)
    
    # Load the image
    original_img = cv2.imread(image_path)
    if original_img is None:
        print(f"Image {image_name} not found, skipping...")
        return None

    # Get bounding box coordinates
    x1, y1, x2, y2 = bounding_box

    # Crop the image
    cropped_img = original_img[int(y1):int(y2), int(x1):int(x2)]

    # Save the cropped image
    cropped_image_name = f"cropped_{image_name}"
    cropped_image_path = os.path.join(output_dir, cropped_image_name)
    cv2.imwrite(cropped_image_path, cropped_img)
    print(f"Cropped image saved: {cropped_image_path}")

    return cropped_image_path

# Specify the frame index
frame_index = 735  # Replace with the actual frame index

# Get the bounding boxes for the specified frame from the grid scores dictionary
bounding_boxes = get_bounding_boxes_by_frame(grid_scores_dict, frame_index)

if bounding_boxes is not None:
    # Crop the image for each bounding box
    for bounding_box in bounding_boxes:
        crop_image(frame_index, bounding_box)
else:
    print(f"No bounding boxes found for frame {frame_index}.")